In [1]:
from text import *
from decimal import Decimal
import string

In [2]:
# Helper Functions

def sentences_from_dataset(file_path):
    """ Read text from a language file and extract sentences. """

    with open(file_path, 'r') as myfile:
        lines=myfile.readlines()
    
    sentences = []

    for line in lines:
        number, sentence = line.split('\t')
        sentences.append(sentence)
    
    return sentences

def clean_text(sentence):
    """ Lowercase the sentence, replace newline character with space and remove punctuation. """

    sentence = sentence.replace('\n', ' ')
    sentence = sentence.lower()
    return ''.join(l for l in sentence if l not in string.punctuation)

We first extract the sentences from our training set for each lanuage and clean them up. ** I am yet to add a description of how the data is stored.**

In [3]:
english_sentences = sentences_from_dataset('data/eng.txt')
french_sentences = sentences_from_dataset('data/fra.txt')
ind_sentences = sentences_from_dataset('data/ind.txt')

Now we can clean each of these sentences. And join them with a single space.

In [4]:
english_sentences = ' '.join([clean_text(s) for s in english_sentences])
french_sentences = ' '.join([clean_text(s) for s in french_sentences])
ind_sentences = ' '.join([clean_text(s) for s in ind_sentences])

Now we will use some test text and clean it up. The final notebook will have more of these possibly in a dict with keys so we can count how many we got correct.

In [5]:
test1_ind = '''Setiap orang berhak mendapat pendidikan. Pendidikan harus gratis, setidak-tidaknya untuk tingkat sekolah rendah dan pendidikan dasar. Pendidikan rendah harus diwajibkan. Pendidikan teknik dan jurusan secara umum harus terbuka bagi semua orang, dan pengajaran tinggi harus secara adil dapat diakses oleh semua orang, berdasarkan kepantasan.'''
test2_eng = '''In the traditional sense a hacker is a person who is extremely interested in exploring the things and recondite workings of any computer system. Most often, hackers are the expert programmers.'''

test1_ind = clean_text(test1_ind)
test2_eng = clean_text(test2_eng)

# Implementation 1: Using modified version of NgramTextModel in text.py

The current implementation is supposed to be used with words. We can use it for letter n-grams by treating each character as a word. We can create a char n-gram for each language in our training set. I have introduced a slight modification in the constructor to support Laplacian smoothing. You can compare the orignal and the modified version.

In [6]:
%psource NgramTextModel

In [7]:
# New Version

class NgramTextModel(CountingProbDist):

    """This is a discrete probability distribution over n-tuples of words.
    You can add, sample or get P[(word1, ..., wordn)]. The method P.samples(n)
    builds up an n-word sequence; P.add and P.add_sequence add data."""

    def __init__(self, n, observation_sequence=[], default=1):
        # In addition to the dictionary of n-tuples, cond_prob is a
        # mapping from (w1, ..., wn-1) to P(wn | w1, ... wn-1)
        CountingProbDist.__init__(self, default=default)
        self.n = n
        self.cond_prob = defaultdict()
        self.add_sequence(observation_sequence)

    # __getitem__, top, sample inherited from CountingProbDist
    # Note they deal with tuples, not strings, as inputs

    def add(self, ngram):
        """Count 1 for P[(w1, ..., wn)] and for P(wn | (w1, ..., wn-1)"""
        CountingProbDist.add(self, ngram)
        if ngram[:-1] not in self.cond_prob:
            self.cond_prob[ngram[:-1]] = CountingProbDist()
        self.cond_prob[ngram[:-1]].add(ngram[-1])

    def add_sequence(self, words):
        """Add each of the tuple words[i:i+n], using a sliding window.
        Prefix some copies of the empty word, '', to make the start work."""
        n = self.n
        words = ['', ] * (n - 1) + words
        for i in range(len(words) - n):
            self.add(tuple(words[i:i + n]))

    def samples(self, nwords):
        """Build up a random sample of text nwords words long, using
        the conditional probability given the n-1 preceding words."""
        n = self.n
        nminus1gram = ('',) * (n-1)
        output = []
        for i in range(nwords):
            if nminus1gram not in self.cond_prob:
                nminus1gram = ('',) * (n-1)  # Cannot continue, so restart.
            wn = self.cond_prob[nminus1gram].sample()
            output.append(wn)
            nminus1gram = nminus1gram[1:] + (wn,)
        return ' '.join(output)



In [8]:
e_ng = NgramTextModel(3, list(english_sentences), default=1)
f_ng = NgramTextModel(3, list(french_sentences),default=1)
i_ng = NgramTextModel(3, list(ind_sentences),default=1)

It is possible to see the most commong occurences.

In [9]:
e_ng.top(5)

[(18862, (' ', 't', 'h')),
 (15290, ('t', 'h', 'e')),
 (13334, ('h', 'e', ' ')),
 (7173, ('e', 'd', ' ')),
 (6920, ('i', 'n', 'g'))]

As we are treating P(l) the prior for language as uniform we ignore this term in our calculations. We can evaluate the probability of P(l | test_text) for each language by multiplying the probability of each n-gram in the test_text. We are dealing with tri-grams here.

In [10]:
def create_ngrams(text, n):
    """ List of ngram tuples that work well with our NGramTextModel """
    return [tuple(text[i:i+n]) for i in range(len(text)-1)]

test_1_ngrams = create_ngrams(test1_ind, 3)
print(test_1_ngrams)

[('s', 'e', 't'), ('e', 't', 'i'), ('t', 'i', 'a'), ('i', 'a', 'p'), ('a', 'p', ' '), ('p', ' ', 'o'), (' ', 'o', 'r'), ('o', 'r', 'a'), ('r', 'a', 'n'), ('a', 'n', 'g'), ('n', 'g', ' '), ('g', ' ', 'b'), (' ', 'b', 'e'), ('b', 'e', 'r'), ('e', 'r', 'h'), ('r', 'h', 'a'), ('h', 'a', 'k'), ('a', 'k', ' '), ('k', ' ', 'm'), (' ', 'm', 'e'), ('m', 'e', 'n'), ('e', 'n', 'd'), ('n', 'd', 'a'), ('d', 'a', 'p'), ('a', 'p', 'a'), ('p', 'a', 't'), ('a', 't', ' '), ('t', ' ', 'p'), (' ', 'p', 'e'), ('p', 'e', 'n'), ('e', 'n', 'd'), ('n', 'd', 'i'), ('d', 'i', 'd'), ('i', 'd', 'i'), ('d', 'i', 'k'), ('i', 'k', 'a'), ('k', 'a', 'n'), ('a', 'n', ' '), ('n', ' ', 'p'), (' ', 'p', 'e'), ('p', 'e', 'n'), ('e', 'n', 'd'), ('n', 'd', 'i'), ('d', 'i', 'd'), ('i', 'd', 'i'), ('d', 'i', 'k'), ('i', 'k', 'a'), ('k', 'a', 'n'), ('a', 'n', ' '), ('n', ' ', 'h'), (' ', 'h', 'a'), ('h', 'a', 'r'), ('a', 'r', 'u'), ('r', 'u', 's'), ('u', 's', ' '), ('s', ' ', 'g'), (' ', 'g', 'r'), ('g', 'r', 'a'), ('r', 'a', 't

Calculating **P(lang | text)**. We use decimal for precision reasons.

In [11]:
from utils import product

prob_lang_dict = dict(
english = product([Decimal(e_ng[trigram]) for trigram in test_1_ngrams]),
indonesian = product([Decimal(i_ng[trigram]) for trigram in test_1_ngrams]),
french = product([Decimal(f_ng[trigram]) for trigram in test_1_ngrams]))

It is now possible to compare these to predict the language:

In [12]:
max(prob_lang_dict, key=prob_lang_dict.get)

'indonesian'

The above workflow nicely wrapped up into a Class. I have left out the procedure of getting the data and cleaning because it may vary depending on the source of data.

In [13]:
class LanguageID:
    """ training_corpus should be a dict of language name as keys and cleaned up strings of text as values. """

    def __init__(self, training_corpus, n=3, smoothing_factor=1):
        self.n = n
        self.languages = training_corpus.keys()
        self.training_ngram_models = {language: NgramTextModel(n, list(text), smoothing_factor) 
                                   for language, text in training_corpus.items()}

    def create_ngrams(self, text):
        """ List of ngram tuples that work well with our NGramTextModel """
        return [tuple(text[i:i+self.n]) for i in range(len(text)-1)]
    
    def calculate_posterior(self, language, test_ngrams):
        """ Posterior sans the P(l) term by taking product of prob of each ngram """
        return product([Decimal(self.training_ngram_models[language][ngram]) for ngram in test_ngrams])

    def predict(self, text):
        test_ngrams = self.create_ngrams(text)
        prob_lang_dict = {language: self.calculate_posterior(language, test_ngrams) for language in self.languages}
        return max(prob_lang_dict, key=prob_lang_dict.get) # Key with max prob
    

In [14]:
predictor = LanguageID(dict(english=english_sentences, indonesian=ind_sentences, french=french_sentences))

In [15]:
predictor.predict(test2_eng)

'english'